<a href="https://colab.research.google.com/github/charlesHsuGG/Bert-Relation-Mapping/blob/master/Bert_Relation_Chin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip
!unzip wwm_cased_L-24_H-1024_A-16.zip

--2020-02-06 06:34:52--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.194.128, 2607:f8b0:4001:c14::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1242589256 (1.2G) [application/zip]
Saving to: ‘wwm_cased_L-24_H-1024_A-16.zip’

wwm_cased_L-24_H-10 100%[===================>]   1.16G   183MB/s    in 6.4s    

2020-02-06 06:34:59 (185 MB/s) - ‘wwm_cased_L-24_H-1024_A-16.zip’ saved [1242589256/1242589256]

Archive:  wwm_cased_L-24_H-1024_A-16.zip
   creating: wwm_cased_L-24_H-1024_A-16/
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: wwm_cased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: wwm_cased_L-24_H-1024_A-16

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)
print(tf.test.gpu_device_name())

2.1.0
/device:GPU:0


In [ ]:
import re
from collections import Counter
import tensorflow_hub as hub
import tensorflow as tf
from tokenization import FullTokenizer     # Still from bert module
from modeling import BertConfig, BertModel
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Google Colab don't need this. FullTokenizer is not updated to tf2.0 yet
tf.gfile = tf.io.gfile

In [ ]:
MODEL_DIR = "wwm_cased_L-24_H-1024_A-16" #@param {type:"string"} ['wwm_cased_L-24_H-1024_A-16']

config_path = "/content/{}/bert_config.json".format(MODEL_DIR)
vocab_path = "/content/{}/vocab.txt".format(MODEL_DIR)

bert_config = BertConfig.from_json_file(config_path)

In [ ]:
class RelationTokebizer(FullTokenizer):

  def tokenize(self, text):
      split_tokens = []
      for sub_text in re.split("(<e[1-9]1>|<e[1-9]2>)", text):
        if re.findall("^(<e[1-9]1>|<e[1-9]2>)", sub_text):
          split_tokens.append(sub_text)
        else:
          for token in self.basic_tokenizer.tokenize(sub_text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
              if sub_token in self.vocab:
                split_tokens.append(sub_token)
              elif _is_whitespace(sub_token):
                split_tokens.append("[unused1]")
              else:
                split_tokens.append("[UNK]")

      return split_tokens

In [ ]:
tokenizer = RelationTokebizer(vocab_path, do_lower_case=True)

**建立切詞器**

In [ ]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [ ]:
def relation_map():
    relation_map = {"UNKNOW":0}
    data_path = "/content/label.txt"
    with open(data_path, 'r') as f:
      for line in f:
          name = line.replace("\n", "")
          relation_map[name] = len(relation_map)
      return relation_map

In [ ]:
relation_map = relation_map()

In [ ]:
relation_map

{'Cause-Effect(e1,e2)': 2,
 'Cause-Effect(e2,e1)': 3,
 'Component-Whole(e1,e2)': 14,
 'Component-Whole(e2,e1)': 15,
 'Content-Container(e1,e2)': 8,
 'Content-Container(e2,e1)': 9,
 'Entity-Destination(e1,e2)': 12,
 'Entity-Destination(e2,e1)': 13,
 'Entity-Origin(e1,e2)': 10,
 'Entity-Origin(e2,e1)': 11,
 'Instrument-Agency(e1,e2)': 4,
 'Instrument-Agency(e2,e1)': 5,
 'Member-Collection(e1,e2)': 16,
 'Member-Collection(e2,e1)': 17,
 'Message-Topic(e1,e2)': 18,
 'Message-Topic(e2,e1)': 19,
 'Other': 1,
 'Product-Producer(e1,e2)': 6,
 'Product-Producer(e2,e1)': 7,
 'UNKNOW': 0}

In [ ]:
relation_type_num = len(relation_map)
max_seq_length=384

In [ ]:
class RBERT(tf.keras.Model):

  def __init__(self, doropout_rate = 0.1, max_seq_length=max_seq_length, bert_config = bert_config, hidden_size=768, num_labels=relation_type_num):
    super(RBERT, self).__init__()
    self.input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
    self.input_word_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="input_word_mask")
    self.segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                        name="segment_ids")
    self.e1_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                        name="e1_mask")
    self.e2_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                        name="e2_mask")
    
    self.bert_layer = BertModel(config=bert_config, name='bert')

    self.cls_fc_droput = tf.keras.layers.Dropout(doropout_rate)
    self.cls_fc_layer = tf.keras.layers.Dense(hidden_size, activation="tanh")
    self.e1_fc_droput = tf.keras.layers.Dropout(doropout_rate)
    self.e1_fc_layer = tf.keras.layers.Dense(hidden_size, activation="tanh")
    self.e2_fc_droput = tf.keras.layers.Dropout(doropout_rate)
    self.e2_fc_layer = tf.keras.layers.Dense(hidden_size, activation="tanh")
    
    self.concat_h = tf.keras.layers.Concatenate(axis=-1)

    self.logit_output = tf.keras.layers.Dropout(doropout_rate)
    self.logit = tf.keras.layers.Dense(num_labels, activation="softmax")



  def call(self, inputs, build = False):
    input_word_ids, input_mask, segment_ids, e1_mask_input, e2_mask_input = inputs
    if not build:
      input_word_ids = self.input_word_ids(input_word_ids)
      input_mask = self.input_word_mask(attention_mask)
      segment_ids = self.segment_ids(token_type_ids)
      e1_mask_input = self.e1_mask(e1_mask)
      e2_mask_input = self.e2_mask(e2_mask)
    pooled_output, sequence_output = self.bert_layer(input_word_ids=input_word_ids,
                                                      input_mask=input_mask,
                                                      input_type_ids=segment_ids)
    e1_h = self.entity_average(sequence_output, e1_mask_input)
    e2_h = self.entity_average(sequence_output, e2_mask_input)
    print(e1_h)
    cls_dropout = self.cls_fc_droput(pooled_output)
    pooled_output = self.cls_fc_layer(cls_dropout)

    e1_dropout = self.e1_fc_droput(e1_h)
    e1_h  = self.e1_fc_layer(e1_dropout)

    e2_dropout = self.e2_fc_droput(e2_h)
    e2_h  = self.e2_fc_layer(e2_dropout)

    concat_h = self.concat_h([pooled_output, e1_h, e2_h])

    drop_output = self.logit_output(concat_h)
    logits = self.logit(drop_output)

    return logits

  def buile_model(self, max_seq_length=max_seq_length):
      input_ids_shape=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32)
      attention_mask_shape=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32)
      token_type_ids_shape=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32)
      e1_mask_shape=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32)
      e2_mask_shape=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32)
      inputs = input_ids_shape, attention_mask_shape, token_type_ids_shape, e1_mask_shape, e2_mask_shape
      return Model(inputs=[inputs], outputs=self.call(inputs, build = True))

  @staticmethod
  def entity_average(sequence_output, e_mask):
    e_mask_unsqueeze = tf.expand_dims(e_mask, 1)
    length_tensor = tf.expand_dims(tf.reduce_sum(e_mask, 1), 1)
    sum_vector = tf.squeeze(tf.matmul(tf.dtypes.cast(e_mask_unsqueeze, tf.float32), sequence_output), axis = 1)
    avg_vector = tf.dtypes.cast(sum_vector, tf.float32) / tf.dtypes.cast(length_tensor, tf.float32)
    return avg_vector

In [ ]:
model = RBERT().buile_model()

Tensor("truediv:0", shape=(None, 1024), dtype=float32)


In [ ]:
import numpy as np

In [ ]:
model_params = {v.name:v for v in model.trainable_variables}
model_roots = np.unique([v.name.split('/')[0] for v in model.trainable_variables])
# print(model_roots)
saved_names = [k for k,v in tf.train.list_variables('/content/wwm_cased_L-24_H-1024_A-16/bert_model.ckpt')]
a_map = {v:v+':0' for v in saved_names}
# print(a_map)
model_roots = np.unique([v.name.split('/')[0] for v in model.trainable_variables])
# print(model_roots)
def transform(x):
    x = x.replace('attention/self','attention')
    x = x.replace('attention','self_attention')
    x = x.replace('attention/output','attention_output')  

    x = x.replace('/dense','')
    x = x.replace('/LayerNorm','_layer_norm')
    x = x.replace('embeddings_layer_norm','embeddings/layer_norm')  

    x = x.replace('attention_output_layer_norm','attention_layer_norm')  
    x = x.replace('embeddings/word_embeddings','word_embeddings/embeddings')

    x = x.replace('/embeddings/','/embedding_postprocessor/')  
    x = x.replace('/token_type_embeddings','/type_embeddings')  
    x = x.replace('/pooler/','/pooler_transform/')  
    x = x.replace('answer_type_output_bias','ans_type/bias')  
    x = x.replace('answer_type_output_','ans_type/')
    x = x.replace('cls/nq/output_','logits/')
    x = x.replace('/weights','/kernel')

    return x
a_map = {k:model_params.get(transform(v),None) for k,v in a_map.items() if k.startswith('bert')}
# print(a_map)
tf.compat.v1.train.init_from_checkpoint(ckpt_dir_or_file='/content/wwm_cased_L-24_H-1024_A-16/bert_model.ckpt',
                                        assignment_map=a_map)

In [ ]:
model.layers[7].trainable = False

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=2e-5)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 384)]        0                                            
______________________________________________________________________________________________

In [ ]:
# def insert_entity_token(sentence, entity_pair):
#   temp_start_word = ""
#   temp_end_word = ""
#   sent_list = list(sentence)
#   for i, ent in enumerate(entity_pair):
#     index = 0
#     temp_sent_list = []
#     for word in sent_list:
#         if index == ent['start_ind'] and index == ent['end_ind'] - 1:
#            temp_sent_list.append("<e"+str(i+1)+"1>")
#            temp_sent_list.append(word)
#            temp_sent_list.append("<e"+str(i+1)+"2>")
#         elif index == ent['start_ind']:
#            temp_sent_list.append("<e"+str(i+1)+"1>")
#            temp_sent_list.append(word)
#         elif index == ent['end_ind'] - 1:
#            temp_sent_list.append(word)
#            temp_sent_list.append("<e"+str(i+1)+"2>")
#         else:
#            temp_sent_list.append(word)
#         if temp_start_word and temp_end_word:
#            if word != temp_start_word or word != temp_end_word:
#               index += 1
#         else:
#            index += 1
#     # print(temp_sent_list)
#     sent_list = temp_sent_list
#     temp_start_word = "<e"+str(i+1)+"1>"
#     temp_end_word = "<e"+str(i+1)+"2>"
#   return "".join(sent_list)

In [ ]:
def convert_relation_type(type_name):
    return relation_map[type_name]

In [ ]:
def convert_to_feature(sentence, max_seq_length = max_seq_length):
    """
    convert to feature
    """
    sentence = sentence.replace("<e1>", "<e11>").replace("</e1>", "<e12>").replace("<e2>", "<e21>").replace("</e2>", "<e22>")
    tokens_a = tokenizer.tokenize(sentence)

    if "<e22>" not in tokens_a:
      print(tokens_a)
    e11_p = tokens_a.index("<e11>")  # the start position of entity1
    e12_p = tokens_a.index("<e12>")  # the end position of entity1
    e21_p = tokens_a.index("<e21>")  # the start position of entity2
    e22_p = tokens_a.index("<e22>")  # the end position of entity2

    tokens_a[e11_p] = "$"
    tokens_a[e12_p] = "$"
    tokens_a[e21_p] = "#"
    tokens_a[e22_p] = "#"

    e11_p += 1
    e12_p += 1
    e21_p += 1
    e22_p += 1

    special_tokens_count = 2 #### for bert
    if len(tokens_a) > max_seq_length - special_tokens_count:
        tokens_a = tokens_a[:(max_seq_length - special_tokens_count)]
    # print(tokens_a)
    stokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    input_word_ids = get_ids(stokens, tokenizer, max_seq_length=max_seq_length)
    input_mask = get_masks(stokens, max_seq_length=max_seq_length)
    segment_ids = get_segments(stokens, max_seq_length=max_seq_length)
    
    e1_mask = [0] * len(input_mask)
    e2_mask = [0] * len(input_mask)

    for i in range(e11_p, e12_p):
        e1_mask[i] = 1
    for i in range(e21_p, e22_p):
        e2_mask[i] = 1

    assert len(input_word_ids) == max_seq_length, "Error with input length {} vs {}".format(len(input_word_ids), max_seq_length)
    assert len(input_mask) == max_seq_length, "Error with attention mask length {} vs {}".format(len(input_mask), max_seq_length)
    assert len(segment_ids) == max_seq_length, "Error with token type length {} vs {}".format(len(segment_ids), max_seq_length)
    
    return input_word_ids, input_mask, segment_ids, e1_mask, e2_mask

In [ ]:
def data_generator(data_path = "/content/train.tsv"):
    with open(data_path, 'r') as f:
      for line in f:
        relation_type, sent = line.split('\t')
        # print(entity_a, entity_b)
        input_features = convert_to_feature(sent)
        # print(relation_type)
        label_id = convert_relation_type(relation_type)
        yield input_features, label_id

In [ ]:
train_ds = tf.data.Dataset.from_generator(data_generator, ((tf.int32, tf.int32, tf.int32, tf.int32, tf.int32), (tf.int32)), args=("/content/train.tsv",))

In [ ]:
test_ds = tf.data.Dataset.from_generator(data_generator, ((tf.int32, tf.int32, tf.int32, tf.int32, tf.int32), (tf.int32)), args=("/content/test.tsv",))

In [ ]:
TRAIN_DATA_SIZE = len([ds for ds in train_ds])
TEST_DATA_SIZE = len([ds for ds in test_ds])

In [ ]:
print(TRAIN_DATA_SIZE)
print(TEST_DATA_SIZE)

8000
2717


In [ ]:
batch_size = 32

In [ ]:
train_size = int(0.7 * TRAIN_DATA_SIZE)
val_size = int(0.3 * TRAIN_DATA_SIZE)


full_dataset = train_ds.shuffle(10, reshuffle_each_iteration = False )
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.take(val_size)

In [ ]:
train_dataset1 = train_dataset.repeat().batch(batch_size, drop_remainder=True)

In [ ]:
def generator():
    while True:
        print("initialize DataSet")
        for inputs, output in train_dataset1:
            yield inputs, output

In [ ]:
val_dataset1 = val_dataset.repeat().batch(batch_size, drop_remainder=True)

In [ ]:
def vaild_generator():
    while True:
      print("initialize DataSet")
      for inputs, output in val_dataset1:
          yield inputs, output

In [ ]:
el_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=5, min_lr=0.001)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = 'model_zero7.{epoch:02d}-{loss:.6f}.hdf5',
                                                   verbose=1,
                                                   save_best_only=True, save_weights_only = True)

In [ ]:
nums_epoch = 30
h = model.fit_generator(generator(), callbacks=[el_stop, reduce_lr, checkpointer], 
                        epochs=nums_epoch, steps_per_epoch=train_size // batch_size, 
                        validation_data = vaild_generator(), validation_steps = val_size//batch_size,
                        use_multiprocessing = False)

initialize DataSet
  ...
    to  
  ['...']
initialize DataSet
  ...
    to  
  ['...']
Train for 175 steps, validate for 75 steps
Epoch 1/30
174/175 [============================>.] - ETA: 3s - loss: 2.6302 - accuracy: 0.1859
Epoch 00001: val_loss improved from inf to 2.40859, saving model to model_zero7.01-2.628163.hdf5
175/175 [==============================] - 879s 5s/step - loss: 2.6282 - accuracy: 0.1864 - val_loss: 2.4086 - val_accuracy: 0.2621
Epoch 2/30
174/175 [============================>.] - ETA: 3s - loss: 2.2617 - accuracy: 0.3111
Epoch 00002: val_loss improved from 2.40859 to 2.15698, saving model to model_zero7.02-2.260736.hdf5
175/175 [==============================] - 867s 5s/step - loss: 2.2607 - accuracy: 0.3111 - val_loss: 2.1570 - val_accuracy: 0.3492
Epoch 3/30
174/175 [============================>.] - ETA: 3s - loss: 2.0090 - accuracy: 0.4018
Epoch 00003: val_loss improved from 2.15698 to 1.97048, saving model to model_zero7.03-2.008777.hdf5
175/175 [=========

In [ ]:
test_dataset1 = test_ds.repeat().batch(batch_size, drop_remainder=True)

def test_generator():
    while True:
      print("initialize DataSet")
      for inputs, output in test_dataset1:
          yield inputs, output

In [ ]:
model.evaluate_generator(test_generator(), steps=TEST_DATA_SIZE//batch_size)

Instructions for updating:
Please use Model.evaluate, which supports generators.
initialize DataSet
  ...
    to  
  ['...']


[0.8839845983755021, 0.71949404]